In [ ]:
# from google.colab import drive
import pandas as pd
import numpy as np
import re

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve,auc
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score
import seaborn as sns
from scipy.stats import ranksums
import pickle

from datetime import datetime
import random
from random import sample
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cluster_num=25
file_name='post_combat_gated_CD4_modify_noHD.csv'
condition_list=['*Infection','*Infection','*Infection','*Ascension','*FollowUp','*Ascension','*FollowUp','*Ascension','*FollowUp']
name_to_select=['inf01','inf02','inf12','inf1-asc01','inf1-fol01','inf2-asc01','inf2-fol01','asc01','fol01']
label_list=[[0,1],[0,2],[1,2],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1]]
phy_6=['143Nd_CD45RA', '167Er_CD197_CCR7','164Dy_CD95', '141Pr_CD196_CCR6', '156Gd_CD183_CXCR3', '149Sm_CD194_CCR4']
infection_select_idx=[3,4,5,6,8]
select_idx = 0# change the index number if needed-  the index matching to name_to_select condition

# TSNE color define
colors = [
"#FF0000",  # Red
"#FFA500",  # Orange
"#FFFF00",  # Yellow
"#008000",  # Green
"#00FFFF",  # Cyan
"#0000FF",  # Blue
"#FF00FF",  # Magenta
"#800080",  # Purple
"#FFC0CB",  # Pink
"#FF4500",  # Orange Red
"#FFD700",  # Gold
"#808000",  # Olive
"#008080",  # Teal
"#000080",  # Navy
"#FF1493",  # Deep Pink
"#FF69B4",  # Hot Pink
"#BA55D3",  # Medium Orchid
"#8A2BE2",  # Blue Violet
"#1E90FF",  # Dodger Blue
"#808080",  # Gray
"#4B0082",  # Indigo
"#7FFF00",  # Chartreuse
"#FFA07A",  # Light Salmon
"#800000",  # Maroon
"#20B2AA",  # Light Sea Green
]

check matrix size:  (38, 224) 38


In [ ]:
def tsne_initial(matrix_length,in_datetime_object,in_PTID,inall_cell_id,incell_label,expression_matrix):
    tsne_cell_label=[]
    sub_trans=np.empty((0,matrix_length), float)
    index_list=list()
    random.seed(in_datetime_object.timestamp())
    for single_sample in in_PTID: #1~n_clusters
        ii = np.where(np.array(inall_cell_id) == single_sample)[0]
        sampled_list = random.sample(list(ii), 1000)
        tsne_cell_label.extend(incell_label[sampled_list])
        index_list.extend(sampled_list)
        sub_trans=np.append(sub_trans, expression_matrix[sampled_list], axis=0)
    tsne = TSNE(n_components=2, perplexity=80,random_state=1024)
    in_all_embedded = tsne.fit_transform(sub_trans)
    return in_all_embedded,tsne_cell_label,index_list

def tsne_all_cluster(set_colors,tsne_df,in_tsne_path):
    sns.palplot(sns.color_palette(set_colors))

    plt.figure(figsize=(26, 26))
    sns.set(font_scale=4)
    sns.set_style(style='white')

    ax=sns.scatterplot(x='TSNE1', y='TSNE2', data=tsne_df, hue='target',palette=sns.color_palette(set_colors),legend=False, s=90)
    # for x_l in list(pca_df['target']):
    #     plt.annotate(str(x_l), pca_df.loc[pca_df['target']==str(x_l),['TSNE1','TSNE2']].mean(),horizontalalignment='center',verticalalignment='center',size=20, weight='bold')

    ax.set_xticks([])
    ax.set_yticks([])
    ax.set(xlabel='TSNE1')
    ax.set(ylabel='TSNE2')
    # plt.savefig(in_tsne_path, dpi=300)
    plt.plot()

def tsne_freq_phenotype(in_Frame,tsne_df,in_all_embedded,in_tsne_path):
    sns.set(font_scale=4)
    sns.set_style(style='white')
    fig, axes = plt.subplots(2,6, figsize=(72, 11), gridspec_kw={'height_ratios': [16,1]})
    for i in range(0,6,1):
        labels_cc=in_Frame.iloc[:,i]
        trans_temp = np.copy(tsne_df['target'])
        for singless in range(0,len(in_Frame),1):
            trans_temp[trans_temp==str(singless+1)]=labels_cc[singless]
        pca_temp = pd.DataFrame(data=in_all_embedded, columns=['TSNE1', 'TSNE2'])
        pca_temp['target']=trans_temp
        ax=sns.scatterplot(ax=axes[0,i],x='TSNE1',y='TSNE2',data=pca_temp,hue='target',palette="viridis_r",legend=False,s=3)#
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xlabel('TSNE1', fontdict=dict(weight='bold'))
        ax.set_ylabel('TSNE2', fontdict=dict(weight='bold'))
        norm = plt.Normalize(pca_temp['target'].min(), pca_temp['target'].max())
        sm = plt.cm.ScalarMappable(cmap="viridis_r", norm=norm)
        sm.set_array([])
        cbar =ax.figure.colorbar(sm,orientation='horizontal',cax=axes[1,i])
        cbar.ax.tick_params(labelsize=50)
        for t in cbar.ax.xaxis.get_major_ticks():
            t.label1.set_fontweight('bold')
        get_phy_name=np.array(in_Frame.columns)[i]
        get_phy_name=get_phy_name.split("_")
        axes[0,i].set_title(get_phy_name[len(get_phy_name)-1], fontdict = { 'weight': 'bold'})
    # plt.savefig(in_tsne_path, dpi=300)
    plt.plot()


def filter_infection(in_dfv_0,in_cell_cluster,in_select_idx):
    if in_select_idx==3 or in_select_idx==4:
        visit_list=np.array(in_dfv_0[in_dfv_0['*Infection'] ==1].index.tolist()) ###

    elif in_select_idx==5 or in_select_idx==6:
        visit_list=np.array(in_dfv_0[in_dfv_0['*Infection'] ==2].index.tolist()) ###

    elif in_select_idx==8:
        visit_list=np.array(in_dfv_0[in_dfv_0['*Infection'] !=2].index.tolist()) ###
    in_dfv_0=in_dfv_0.iloc[visit_list,:]
    in_cell_cluster=in_cell_cluster[visit_list]
    in_dfv_0=in_dfv_0.reset_index(drop=True)
    in_p_PTID=np.unique(in_dfv_0['*PTID'])
    return in_dfv_0,in_cell_cluster,in_p_PTID

In [ ]:
#random seeds define
timestamp_string = '2023-02-03T14::12::53'
format = '%Y-%m-%dT%H::%M::%S'
datetime_object = datetime.strptime(timestamp_string, format)
random.seed(datetime_object.timestamp())


#load file
save_name=file_name.split('_')[3]
df = pd.read_csv(file_name) # path for raw data

all_cell_id=df['*PTID']
unique_ID=np.unique(df['*PTID'])
select_cell_per_ID=list()

#downsample per participate
for u_id in unique_ID:
    id_idx=np.where(np.array(all_cell_id) == u_id)[0]
    downsample_id=sample(list(id_idx), 15000)
    select_cell_per_ID.extend(downsample_id)

df=df.iloc[select_cell_per_ID]
df=df.reset_index(drop=True)
all_cell_id=df['*PTID']
unique_ID=np.unique(df['*PTID'])

get_column=df.columns

#pick frequency features
phenotype=np.array([16, 4, 3, 21, 10, 23])

features_name=[]
for i in range(0,cluster_num,1):
    features_name.append("cluster "+str(i+1)+": phenotypic markers")

pick_column=list(np.arange(0,31,1))


dfv_0_arr=np.array(df)
# arcsinh transfer
dm_0_trans = np.arcsinh(1./5 * dfv_0_arr[:,pick_column])
dm_to_df=pd.DataFrame(data=dm_0_trans,columns=get_column[pick_column])

#select functional features
for single_label in list(phenotype):
    pick_column.remove(single_label)
pick_column=np.array(pick_column)

# (1) Automated gating
# Kmean by frequency features
dm_0_trans = np.array(dm_to_df[get_column[phenotype]])
kmeans = KMeans(n_clusters=cluster_num, random_state=0).fit(dm_0_trans)
cluster_centers_arr=np.array(kmeans.cluster_centers_)
Frame=pd.DataFrame(cluster_centers_arr, columns = phy_6)
cell_label=kmeans.labels_

# plot TSNE (Figure S3A-B,D-E)
matrix_len=len(get_column[phenotype])
# TSNE initialize
all_embedded,sub_cell_label,idx_lst=tsne_initial(matrix_len,datetime_object,unique_ID,all_cell_id,cell_label,dm_0_trans)

pca_df = pd.DataFrame(data=all_embedded, columns=['TSNE1', 'TSNE2'])
pca_df['target']=[str(x+1) for x in sub_cell_label]
pca_df['index']=idx_lst

# plot TNSE for all cluster (figure 3B 3E)
tsne_plot_path= save_name + "_allcluster.png"
tsne_all_cluster(colors,pca_df,tsne_plot_path)

#plot TNSE for frequency features  (figure 3A 3D)
tsne_plot_path = save_name + "_markers_phenotype.png"
tsne_freq_phenotype(Frame,pca_df,all_embedded,tsne_plot_path)


In [ ]:
# (2) Frequency Features engineering
# define enrollment or one month
cell_cluster=kmeans.labels_
visit_list=np.array(df[(df['*Visit'] ==0)].index.tolist()) ### |(df['*Visit'] ==2)
cluster_matrix = np.empty((0,cluster_num), float)
dfv_0=df.iloc[visit_list,:]
cell_cluster=cell_cluster[visit_list] ###
dfv_0=dfv_0.reset_index(drop=True)
p_PTID=np.unique(dfv_0['*PTID'])

# filter data if the comparsion only look into specific condition
if select_idx in infection_select_idx:
    dfv_0,cell_cluster,p_PTID=filter_infection(dfv_0,cell_cluster,select_idx)


#create vector per participant
condition_select=condition_list[select_idx]
infection_dfv_0=np.array(dfv_0[condition_select])
label_for_condition=label_list[select_idx]
cluster_label=[]
get_PTID=[]
for iid in p_PTID:
    freq=[]
    ii = np.where(dfv_0['*PTID'] == iid)[0]
    # print(iid,np.unique(np.array(infection_dfv_0[ii])))
    get_val=np.unique(np.array(infection_dfv_0[ii]))[0]

    if get_val in label_for_condition: #or get_val==label_for_condition[2]
        get_PTID.append(iid)
        if (len(label_for_condition)==3):
            if get_val==label_for_condition[1] or get_val==label_for_condition[2]: # or get_val==label_for_condition[2]
                cluster_label.append(1)
            elif get_val==label_for_condition[0]:
                cluster_label.append(0)
        elif(len(label_for_condition)==2):
            if get_val==label_for_condition[1]:
                cluster_label.append(1)
            elif get_val==label_for_condition[0]:
                cluster_label.append(0)
        for cluster_i in range(0,cluster_num,1):
            count=0
            for index_ii in ii:
                if (cluster_i)==cell_cluster[index_ii]:##change cluster number
                    count+=1
            freq.append(count)
        freq_feature=np.array(freq)/sum(freq)
        all_marker=np.array(freq_feature)
        cluster_matrix = np.append(cluster_matrix, [all_marker], axis=0)
pheno_func_matrix = pd.DataFrame(data=cluster_matrix, columns=features_name)
# pheno_func_matrix.to_csv("check_matrix.csv")

In [ ]:
# (3) Random forest training
## Random Forest Model
gini_matrix = np.empty((0,len(features_name)), float)


comparison_pred=[] ###
comparison_true=[] ###
comparison_auc=[]
random_state = np.random.RandomState(0)
for i in range(0,30,1):
    clf = RandomForestClassifier(random_state=random_state)
    cv = StratifiedKFold(n_splits=5,shuffle=True)
    single_time_prob=[]
    single_time_true=[]

    for train,test in cv.split(cluster_matrix,np.array(cluster_label)):
        clf.fit(cluster_matrix[train],np.array(cluster_label)[np.array(train)])
        gini=clf.feature_importances_
        gini_matrix = np.append(gini_matrix, [gini], axis=0)
        lr_probs = clf.predict_proba(cluster_matrix[test])
        lr_probs = lr_probs[:, 1]
        comparison_pred+=list(lr_probs) ###
        comparison_true+=list(np.array(cluster_label)[np.array(test)]) ###
        single_time_prob+=list(lr_probs)
        single_time_true+=list(np.array(cluster_label)[np.array(test)])
    fpr, tpr, t = roc_curve(single_time_true, single_time_prob)
    comparison_auc.append(auc(fpr, tpr))

## AUC Result .csv
auc_csv_save = pd.DataFrame(data=comparison_auc, columns=['AUC_result'])
# auc_csv_save.to_csv('CD4EnInf02_aucresult.csv')

# AUC record for ploting the AUC curve
# dict = {'pred':comparison_pred,'true':comparison_true}
# f = open("CD4EnInf02_auc.pkl","wb")
# pickle.dump(dict,f)
# f.close()



# Features Importance Plot .png
plt.figure(figsize=[40,120])
gini_matrix=np.mean(gini_matrix, axis=0)
sorted_idx = gini_matrix.argsort()
plt.barh(np.array(features_name)[sorted_idx], gini_matrix[sorted_idx])
plt.xlabel("Random Forest Feature Importance")
plt.title(save_name+" "+condition_select+" "+str(label_for_condition))
# plt.savefig('CD4EnInf01_featureimportance.png', dpi=300)
plt.plot()

## Features Rank .csv
features_rank=np.argsort(np.argsort(-gini_matrix[sorted_idx]))+1
feature_rank_csv = pd.DataFrame(data=np.array(features_name)[sorted_idx], columns=['features title'])
feature_rank_csv['gini_score']=gini_matrix[sorted_idx]
feature_rank_csv['rank']=features_rank
# feature_rank_csv.to_csv('CD4EnInf01_featuresrank.csv')


In [ ]:
# (4) Gini feature importance analysis
## All Features Rank .csv

def plot_boxplot(in_sst,in_df_total,in_cluster_label,in_cluster_num):
    one_marker=in_df_total.iloc[:,np.array(in_sst)]
    one_marker_arr=np.array(one_marker)
    get_pvalue=[]
    get_compare_uplow=[]
    flipped=False # define if it need flip or not
    """
    example of turning fliped
    Compare to the control group, the "up" and "low" should be decide from:
    in camparsion uninf(get_value = 0) vs CT only(get_value = 1), if let uninf be control group, fliped = False
    in camparsion CT only(get_value = 0) vs Coinf(get_value = 1), if let CT only be control group, fliped = False
    in camparsion Endo-(get_value = 0) vs Endo+(get_value = 1), if let Endo+ be control group, fliped = True
    in camparsion FU-(get_value = 0) vs FU+(get_value = 1), if let FU+ be control group, fliped = True
    (basically if you want label 0 to be control group, set fliped to be False. if you want label 1 to be control group, set fliped to be True)
    """
    infection_dfv_0=np.array(in_cluster_label)
    for cluster_i in range(0,in_cluster_num,1):
        data_T=[]
        data_F=[]
        for iid in range(0,len(in_cluster_label),1):
            get_val=infection_dfv_0[iid]
            if get_val==1:
                data_T.append(one_marker_arr[iid][cluster_i])
            elif get_val==0:
                data_F.append(one_marker_arr[iid][cluster_i])
        if not flipped: # compare to contrel group(label=0)
            if np.mean(data_T)>np.mean(data_F): # save "up" if data_T->label 1 > data_F->label 0
                get_compare_uplow.append('up')
            else:
                get_compare_uplow.append('low')
        else:  # compare to contrel group(label=1)
            if np.mean(data_T)>np.mean(data_F): # save "low" if data_T->label 1 > data_F->label 0
                get_compare_uplow.append('low')
            else:
                get_compare_uplow.append('up')
        get_pvalue.append(ranksums(data_T, data_F).pvalue)
    return get_pvalue,get_compare_uplow



cluster_num=0
for visited in features_name:
  if "phenotypic markers" in visited:
    cluster_num+=1

gini_name_lst=np.array(feature_rank_csv["features title"])
cluster_number_lst,arrow_compare_to_ctrl,total_p=[],[],[]
df_total=pd.DataFrame(data=cluster_matrix, columns=features_name)
pvalue_csv=pd.DataFrame(data=[str(i+1) for i in range(0,25,1)], columns=['cluster'])
upper_lower_csv=pd.DataFrame(data=[str(i+1) for i in range(0,25,1)], columns=['cluster'])


sst=[]
for i in range(0,cluster_num,1):
    sst.append(i)
get_p,get_compare_up_low=plot_boxplot(sst,df_total,cluster_label,cluster_num)
pvalue_csv['phenotype']=get_p
upper_lower_csv['phenotype']=get_compare_up_low



unique_name=[re.split(": | ",i)[2] for i in gini_name_lst]
unique_name=list(np.unique(unique_name))
features_idx_dict={}

for cluster_nam in range(0,len(gini_name_lst),1):
    get_per_name=re.split(": | ",gini_name_lst[cluster_nam])[1:3]
    cluster_number_lst.append(get_per_name[0])
    if get_per_name[1] not in features_idx_dict:
        temp_list=[]
    else:
        temp_list=features_idx_dict[get_per_name[1]]
    temp_list.append(cluster_nam)
    features_idx_dict[get_per_name[1]]=temp_list

    if get_per_name[1]=='phenotypic':
        arrow_compare_to_ctrl.append(upper_lower_csv['phenotype'][list(upper_lower_csv['cluster']).index(get_per_name[0])])
        total_p.append(pvalue_csv['phenotype'][list(pvalue_csv['cluster']).index(get_per_name[0])])
    else:
        arrow_compare_to_ctrl.append(upper_lower_csv[get_per_name[1]][list(upper_lower_csv['cluster']).index(get_per_name[0])])
        total_p.append(pvalue_csv[get_per_name[1]][list(pvalue_csv['cluster']).index(get_per_name[0])])
feature_rank_csv['p_value']=total_p
feature_rank_csv['arrow_compare_to_ctrl']=arrow_compare_to_ctrl

Functional_Marker,Avg_Gini,Gini_score_list,Cluster,P_Value,Upper_or_Lower=[],[],[],[],[],[]

for feature_nam in unique_name:
    Functional_Marker.append(feature_nam)
    Avg_Gini.append(np.mean(np.array(feature_rank_csv['gini_score'])[features_idx_dict[feature_nam]]))
    get_sort_list=np.argsort(np.array(feature_rank_csv['p_value'])[features_idx_dict[feature_nam]])

    get_temp=np.array(feature_rank_csv['p_value'])[features_idx_dict[feature_nam]]
    get_sort_temp=get_temp[get_sort_list]
    keep_idx=1
    for get_idxs in range(0,len(get_sort_temp),1):
        if get_sort_temp[get_idxs]>0.05:
            keep_idx=get_idxs
            break

    P_Value.append(get_temp[get_sort_list][:keep_idx])
    get_temp=np.array(cluster_number_lst)[features_idx_dict[feature_nam]]
    Cluster.append(get_temp[get_sort_list][:keep_idx])

    get_temp=np.array(feature_rank_csv['gini_score'])[features_idx_dict[feature_nam]]
    Gini_score_list.append(get_temp[get_sort_list][:keep_idx])

    get_temp=np.array(feature_rank_csv['arrow_compare_to_ctrl'])[features_idx_dict[feature_nam]]
    Upper_or_Lower.append(get_temp[get_sort_list][:keep_idx])


get_sort_list=list(reversed(np.argsort(Avg_Gini)))
Functional_Marker=np.array(Functional_Marker)[get_sort_list]
Avg_Gini=np.array(Avg_Gini)[get_sort_list]
Gini_score_list=[Gini_score_list[i] for i in get_sort_list]
Cluster=[Cluster[i] for i in get_sort_list]
P_Value=[P_Value[i] for i in get_sort_list]
Upper_or_Lower=[Upper_or_Lower[i] for i in get_sort_list]


new_Functional_Marker,new_Avg_Gini,new_Gini_score_list,new_Cluster,new_P_Value,new_Upper_or_Lower=list(),list(),list(),list(),list(),list()
for i_fm in range(0,len(Functional_Marker),1):
    for j_pv in range(0,len(P_Value[i_fm]),1):
        new_Functional_Marker.append(Functional_Marker[i_fm])
        new_Avg_Gini.append(round(Avg_Gini[i_fm],4))
        new_Gini_score_list.append(round(Gini_score_list[i_fm][j_pv],4))
        new_Cluster.append(Cluster[i_fm][j_pv])
        new_P_Value.append(P_Value[i_fm][j_pv])
        new_Upper_or_Lower.append(Upper_or_Lower[i_fm][j_pv])



p_range=[[0,0.0001],[0.0001,0.001],[0.001,0.01],[0.01,0.05],[0.05,1]]
p_range_label=['****P<0.0001','***P<0.001','**P<0.01','*P<0.05','None']
newp_label=[]
for every_p_score in new_P_Value:
    for in_p_range in range(0,len(p_range),1):
        if every_p_score>=p_range[in_p_range][0] and every_p_score<p_range[in_p_range][1]:
            newp_label.append(p_range_label[in_p_range])

final_csv=pd.DataFrame(data=new_Functional_Marker, columns=['Marker'])
final_csv['Avg Gini Score']=new_Avg_Gini
final_csv['Gini Score']=new_Gini_score_list
final_csv['Cluster']=new_Cluster
final_csv['P_Value']=newp_label
final_csv['Compared to Uninf']=new_Upper_or_Lower # custom name each time
# final_csv.to_csv('CD4EnInf01_allfeaturesrank.csv',index=False)
print(final_csv)

In [ ]:
# Signed Gini score heatmaps (Fig S3H, S4B)
heatmap_pkl ={}
heatmap_feature_name = []
heatmap_matrix = np.empty((0,cluster_num), float)
clunter_num=[int(re.split(": | ",i)[1]) for i in feature_rank_csv['features title']]
sub_feature_name=[re.split(": | ",i)[2] for i in feature_rank_csv['features title']]
feature_rank_modify=feature_rank_csv.copy()
feature_rank_modify["Marker"]=sub_feature_name
feature_rank_modify["Cluster"]=clunter_num
select_final_csv=feature_rank_modify[feature_rank_modify['Marker']=='phenotypic']
select_final_csv=select_final_csv.sort_values(by=['Cluster'])
dir_list=[-1 if i_dir=='low' else 1 for i_dir in select_final_csv['arrow_compare_to_ctrl']]
correct_score_list=[dir_list[i_dir]*list(select_final_csv['gini_score'])[i_dir] for i_dir in range(0,len(select_final_csv['arrow_compare_to_ctrl']),1)]
heatmap_matrix = np.append(heatmap_matrix, [correct_score_list], axis=0)
heatmap_feature_name.append('Frequency')

heatmap_data=pd.DataFrame(heatmap_matrix,index=heatmap_feature_name, columns=[str(i+1) for i in range(0,25,1)])
heatmap_pkl[name_to_select[select_idx]]=heatmap_data
# f_hm = open("heatmap_df.pkl","wb")
# pickle.dump(heatmap_pkl,f_hm)
# f_hm.close()